In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
from tqdm import tqdm

# Define the headers
header = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36' ,'Accept-Language': 'en-US, en;q=0.5'})

In [2]:
url = 'https://www.magicbricks.com/ready-to-move-flats-in-mumbai-pppfs'

In [3]:
web = requests.get(url , header)

In [4]:
soup = BeautifulSoup(web.text , 'html.parser')

In [10]:
d={'bedrooms':[],'balcony':[],'sqft':[],'floor':[],'facing':[],'furniture':[],'age':[],'location':[],'price':[]}

In [6]:
links=[]
for j in tqdm(range(1,1000)):
    header = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36' ,'Accept-Language': 'en-US, en;q=0.5'})
    url = f'https://www.magicbricks.com/ready-to-move-flats-in-mumbai-pppfs/page-{j}'
    web = requests.get(url , header)
    soup = BeautifulSoup(web.text , 'html.parser')
    
    sqft=soup.find('div' ,class_='mb-srp__card__summary--value')
    

    for i in range(1,30):
        script_tag = soup.findAll('script', attrs={'type': 'application/ld+json'})[i]
        script_text = script_tag.text
        url_match = re.search(r'"url":"(https:\/\/www\.magicbricks\.com\/propertyDetails\/[^"]+)"', script_text)
        if url_match:
            url = url_match.group(1)      
            links.append(url)
        else:
            continue
                   

100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [23:37<00:00,  1.42s/it]


In [7]:
len(links)

10109

In [11]:
for i in tqdm(links,'Scraping:'):
    web=requests.get(i,header)
    soup=BeautifulSoup(web.content , 'html.parser')

    a=soup.find_all('div', class_='mb-ldp__dtls__body__list--value')
    
    b=soup.findAll('li', class_='mb-ldp__dtls__body__summary--item')

    bed=[]
    ##bath=[]
    balcony=[]
    flr=[]
    facing=[]
    furniture=[]
    age=[]

    sqft=soup.find('div', class_='mb-ldp__dtls__body__list--value')
    location=soup.find('a',class_='mb-ldp__dtls__title--link')
    price=soup.find('div', class_='mb-ldp__dtls__price')
    
    for i in range(len(a)):
        if 'Floors' in a[i].text:
            flr.append(a[i].text.strip())
        elif 'years' in a[i].text:
            age.append(a[i].text.strip())
        elif 'furnished'in a[i].text.lower():
            furniture.append(a[i].text.strip())
        elif 'East' in a[i].text or 'West' in a[i].text or 'North' in a[i].text or 'South' in a[i].text:
            facing.append(a[i].text.strip())
            
    for i in range(len(b)):
        if 'Bed' in b[i].text:
            bed.append(b[i].text.strip())
        #elif 'Bath' in b[i].text:
          #  bath.append(b[i].text.strip())
        elif 'Balconies' in b[i].text:
            balcony.append(b[i].text.strip())


    bed = bed if bed else 'NA' 
    #bath = bath if bath else 'NA'  
    balcony = balcony if balcony else 'NA' 
    flr= flr if flr else 'NA'
    facing = facing if facing else 'NA' 
    furniture = furniture if furniture else 'NA'
    age = age if age else 'NA' 
    sqft = sqft.text.strip()[:8]  if sqft else 'NA' 
    location = location.text.strip() if location else 'NA'
    price = price.text.strip() if price else 'NA'
    
            
    d['bedrooms'].append(bed)
    #d['bathrooms'].append(bath)
    d['balcony'].append(balcony)
    d['floor'].append(flr)
    d['facing'].append(facing)
    d['furniture'].append(furniture)
    d['age'].append(age)
    d['sqft'].append(sqft)
    d['location'].append(location)
    d['price'].append(price)
    

Scraping:: 100%|███████████████████████████████████████████████████████████████| 10109/10109 [3:16:45<00:00,  1.17s/it]


In [12]:
df=pd.DataFrame(d)

In [13]:
df

,bedrooms,balcony,sqft,floor,facing,furniture,age,location,price
0,[1Bed],NA,430sqfts,[22 (Out of 22 Floors)],[East],[Unfurnished],NA,"Malad East, Mumbai",₹95.0 Lac
1,[2Beds],NA,845sqfts,[4 (Out of 8 Floors)],NA,[Furnished],NA,"Andheri West, Mumbai",₹1.95 Cr
2,NA,NA,1650sqft,[1 (Out of 7 Floors)],NA,[Furnished],NA,"Juhu, Mumbai",₹7.50 Cr
3,NA,NA,2700sqft,[2 (Out of 10 Floors)],[East],[Furnished],NA,"Walkeshwar, Mumbai",₹27.0 Cr
4,NA,NA,1300sqft,[10 (Out of 11 Floors)],[West],[Semi-Furnished],NA,"Borivali West, Mumbai",₹5.0 Cr
...,...,...,...,...,...,...,...,...,...
10104,[2Beds],NA,510sqfts,[18 (Out of 18 Floors)],[West],[Unfurnished],[Less than 5 years],"Tagore Nagar, Mumbai",₹98.0 Lac
10105,[2Beds],NA,840sqfts,[Ground (Out of 4 Floors)],NA,[Semi-Furnished],NA,"Vile Parle East, Mumbai",₹1.50 Cr
10106,[2Beds],NA,575sqfts,[1 (Out of 4 Floors)],[East],[Semi-Furnished],[10 to 15 years],"Mira Road, Mumbai",₹68.5 Lac
10107,[1Bed],NA,368sqfts,[6 (Out of 6 Floors)],[South],[Unfurnished],[15 to 20 years],"Goregaon East, Mumbai",₹1.10 Cr


In [14]:
df.to_csv('tst_house.csv')

In [ ]:
df['bedrooms']=df['bedrooms'].apply(lambda x : re.sub('[^0-9]','', str(x)) if x!='NA' else 'NA')

# trying on internal link (soup2) to see if we are scraping the right things

In [9]:
url2='https://www.magicbricks.com/propertyDetails/1-BHK-450-Sq-ft-Multistorey-Apartment-FOR-Sale-Naigaon-West-in-Mumbai&id=4d423339373930353631'

In [10]:
web2 = requests.get(url2 , header)
soup2 = BeautifulSoup(web2.content , 'html.parser')

In [12]:
soup2.find('a',class_='mb-ldp__dtls__title--link').text.strip()

'Naigaon West, Mumbai'

In [23]:
soup2.find('a', class_='mb-ldp__dtls__body__list--value')

In [187]:
flr=[]
facing=[]
furniture=[]
age=[]

a=soup2.find_all('div', class_='mb-ldp__dtls__body__list--value')

sqft=soup2.find('div', class_='mb-ldp__dtls__body__list--value').text.strip()[:8]

for i in range(len(a)):
    if 'Floors' in a[i].text:
        flr.append(a[i].text)
    elif 'years' in a[i].text:
        age.append(a[i].text)
    elif 'furnished'in a[i].text.lower():
        furniture.append(a[i].text)
    elif 'East' in a[i].text or 'West' in a[i].text:
        facing.append(a[i].text)


In [189]:
flr

['1 (Out of 7 Floors)']

In [217]:
soup2.findAll('li', class_='mb-ldp__dtls__body__summary--item')[1].text

True

In [222]:
bed=[]
bath=[]
balcony=[]

b=soup2.findAll('li', class_='mb-ldp__dtls__body__summary--item')

for i in range(len(main2)):
    if 'Bed' in b[i].text:
        bed.append(b[i].text)
    elif 'Bath' in b[i].text:
        bath.append(b[i].text)
    elif 'Balconies' in b[i].text:
        balcony.append(b[i].text)


In [225]:
balcony

[]

In [226]:
soup2.find('div', class_='mb-ldp__dtls__body__list--value').text.strip()[:8] 

'500sqfts'